# 11.1 파이썬 문화에서의 인터페이스와 프로토콜

* 인터페이스 : 시스템에서 어떤 역할을 할 수 있게 해주는 객체의 공개 메서드의 일부
* 프로토콜 : 어떤 역할을 완수하기 위한 메서드 집합으로서의 인터페이스

# 11.2 파이썬은 시퀀스를 찾아낸다

다음 `Foo` 클래스는 `abc.Sequence`를 상속하지 않으며, 시퀀스 프로토콜 메서드 중 `__getitem__()` 메서드만 구현한다.

In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0,30,10)[pos]

In [2]:
f = Foo()
f[1]

10

In [3]:
for i in f: print(i)

0
10
20


In [4]:
20 in f

True

`__iter__()` 메서드가 없지만 대체 수단인 `__getitem__()` 메서드가 구현되어 있으므로 반복, `in` 연산자 등이 작동한다.

1장에서 구현한 `FrenchDeck` 클래스도 `abc.Sequence`를 상속하지 않지만, 시퀀스 프로토콜의 `__getitem__()`과 `__len__()` 메서드를 구현한다.

In [5]:
!cat frenchdeck.py

import collections

Card = collections.namedtuple('Card', ['rank','suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                      for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, pos):
        return self._cards[pos]


# 11.3 런타임에 프로토콜을 구현하는 멍키 패칭

`FrenchDeck` 클래스에 `shuffle()` 메서드를 구현할 필요가 없는 것은 `random.shuffle()` 함수가 시퀀스 객체 안의 항목들을 섞어주기 때문이다. 그러나 다음과 같이 입력하면 예외가 발생하는데, 이는 `FrenchDeck` 객체가 할당을 지원하지 않기 때문이다.

In [9]:
from random import shuffle
from frenchdeck import FrenchDeck
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

파이썬은 동적 언어이므로 코드를 대화형 콘솔에서 실행하는 동안에도 이 문제를 수정할 수 있다.

In [11]:
def set_card(deck, pos, card):
    deck._cards[pos] = card
    
FrenchDeck.__setitem__ = set_card

In [12]:
shuffle(deck)

In [13]:
deck[:5]

[Card(rank='4', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='2', suit='hearts'),
 Card(rank='A', suit='diamonds'),
 Card(rank='7', suit='clubs')]

이런 방법을 멍키 패칭(소스 코드를 건드리지 않고 런타임에 클래스나 모듈을 변경하는 행위)라고 한다.

# 11.4 알렉스 마르텔리의 물새

ABC...?

# 11.5 ABC 상속하기

`FrenchDeck2`를 `colletions.MutableSequence`의 서브클래스로 선언한다.

In [16]:
import collections

Card = collections.namedtuple('Card', ['rank','suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                      for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    # MutableSequence를 상속하므로 이 클래스의 추상 메서드인 
    # __delitem__() 도 구현해야함
    def __delitem__(self, position):
        del self._cards[position]
        
    # insert() 또한 추상 메서드
#     def insert(self, position, value):
#         self._cards.insert(position, value)

파이썬은 모듈을 로딩하거나 컴파일할 때가 아니라 실제로 객체를 생성할 때 추상 메서드의 구현 여부를 확인한다.

In [17]:
frenchdeck2 = FrenchDeck2()

TypeError: Can't instantiate abstract class FrenchDeck2 with abstract methods insert

따라서 쓰지 않더라도 `__delitem__()`과 `insert()` 메서드를 구현해야 한다.

# 11.6 표준 라이브러리의 ABC

대부분의 ABC는 `collections.abc` 모듈에 정의되어 있다. 어떤 ABC들이 있는지 살펴보자.

## 11.6.1 collections.abc의 ABC

collections.abc에 들어 있는 ABC에 대한 UML 다이어그램

![collections.abc](https://user-images.githubusercontent.com/33891164/39363928-e9ba4520-49e0-11e8-9cc5-273134e16df4.png)

## 11.6.2 ABC의 숫자탑

`numbers` 패키지는 다음과 같이 계층 구조로 이루어져 있다. `Number`가 최상위 슈퍼클래스이며, `Integral`까지 내려간다.

* Number
* Complex
* Real
* Rational
* Integral

정수형인지 검사해야 하는 경우 `isinstance(x, numbers.Integral)`을 이용하면 된다.

# 11.7 ABC의 정의와 사용

